### %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
### Topic: Cdiscount Image Classification
### Author: Siddharth Chandrasekhar
### Date: November 2019
### %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

# **Importing Packages**

In [0]:
import io
import bson                       
import matplotlib.pyplot as plt
from skimage.data import imread  
import multiprocessing as mp    
import keras
import tensorflow as tf

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict

In [633]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Importing all pytorch related libraries

import torch
import torch.nn as nn
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms

# **Reading and Processing Category File**

In [0]:
catcsv=pd.read_csv('/content/drive/Shared drives/ID576/cdiscount-image-classification-challenge/category_names.csv')

In [0]:
catcsv=catcsv.drop(['category_level2'], axis=1)

In [0]:
catcsv=catcsv.drop(['category_level3'], axis=1)

In [0]:
uniquecat=catcsv['category_level1'].unique()

In [0]:
catcsv.head()

,category_id,category_level1
0,1000021794,ABONNEMENT / SERVICES
1,1000012764,AMENAGEMENT URBAIN - VOIRIE
2,1000012776,AMENAGEMENT URBAIN - VOIRIE
3,1000012768,AMENAGEMENT URBAIN - VOIRIE
4,1000012755,AMENAGEMENT URBAIN - VOIRIE


# Reading Data from BSON File

In [0]:
import bson

data = bson.decode_file_iter(open('/content/drive/Shared drives/IDS576/cdiscount-image-classification-challenge/train.bson', 'rb'))

# Mapping category Names from Category file to Train Data

**Mapping Category names to the first batch of files**

In [0]:
import itertools

temp = itertools.islice(data,0,1000)

In [0]:
data1=pd.DataFrame(temp)

**Adding a column with empty values**

In [0]:
data1['Category_name']=np.nan

In [0]:
data1.head() #TO check if the Category_names are empty

,_id,imgs,category_id,Category_name
0,0,[{'picture': b'\xff\xd8\xff\xe0\x00\x10JFIF\x0...,1000010653,NaN
1,1,[{'picture': b'\xff\xd8\xff\xe0\x00\x10JFIF\x0...,1000010653,NaN
2,2,[{'picture': b'\xff\xd8\xff\xe0\x00\x10JFIF\x0...,1000004079,NaN
3,3,[{'picture': b'\xff\xd8\xff\xe0\x00\x10JFIF\x0...,1000004141,NaN
4,4,[{'picture': b'\xff\xd8\xff\xe0\x00\x10JFIF\x0...,1000015539,NaN


In [0]:
#Code to map category names to our first batch
k=-1
for i in data1['category_id']:
  k=k+1
  z=0
  for j in catcsv['category_id']:
    if i==j:
      data1['Category_name'][k]=catcsv['category_level1'][z]
      break
    z=z+1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [0]:
data1.head() #Now our data1 has the category names

,_id,imgs,category_id,Category_name
0,0,[{'picture': b'\xff\xd8\xff\xe0\x00\x10JFIF\x0...,1000010653,TELEPHONIE - GPS
1,1,[{'picture': b'\xff\xd8\xff\xe0\x00\x10JFIF\x0...,1000010653,TELEPHONIE - GPS
2,2,[{'picture': b'\xff\xd8\xff\xe0\x00\x10JFIF\x0...,1000004079,INFORMATIQUE
3,3,[{'picture': b'\xff\xd8\xff\xe0\x00\x10JFIF\x0...,1000004141,INFORMATIQUE
4,4,[{'picture': b'\xff\xd8\xff\xe0\x00\x10JFIF\x0...,1000015539,BRICOLAGE - OUTILLAGE - QUINCAILLERIE


**Defining function to map category names to the rest of the batch and appending to Data1**

In [0]:
num=100
seq=1000

In [0]:
for i in range(num):
    temp1=itertools.islice(data,0,seq)
    k=-1
    print(i)
    data2=pd.DataFrame(temp1)
    data2['Category_name']=np.nan
    for l in data2['category_id']:
      k=k+1
      z=0
      for j in catcsv['category_id']:
        if l==j:
          data2['Category_name'][k]=catcsv['category_level1'][z]
          break
        z=z+1
    data1=data1.append(data2, ignore_index= True)

0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [0]:
data1.shape

(101000, 4)

**Making Image Tensors by converting Image file into bytes**

In [0]:
tensor_pic =[]

for i in range(len(data1)):

    picture = imread(io.BytesIO(data1['imgs'][i][0]['picture']))
    tensor_pic.append(picture)

In [0]:
data1['imgs_tensor']=tensor_pic

# **Storing and reading the Data from Pickle**

In [0]:
data2_1 = data1[0:25000]
data2_1.to_pickle('/content/drive/My Drive/IDS 576/Project/file1.pkl')

In [0]:
data2_2 = data1[25000:50000]
data2_2.to_pickle('/content/drive/My Drive/IDS 576/Project/file2.pkl')

In [0]:
data2_3 = data1[50000:75000]
data2_3.to_pickle('/content/drive/My Drive/IDS 576/Project/file3.pkl')

In [0]:
data2_4 = data1[75000:100000]
data2_4.to_pickle('/content/drive/My Drive/IDS 576/Project/file4.pkl')

In [0]:
import pickle

In [0]:
p1 = pd.read_pickle('/content/drive/My Drive/IDS 576/Project/file1.pkl')

In [0]:
p2 = pd.read_pickle('/content/drive/My Drive/IDS 576/Project/file2.pkl')

In [0]:
p3 = pd.read_pickle('/content/drive/My Drive/IDS 576/Project/file3.pkl')

In [0]:
p4 = pd.read_pickle('/content/drive/My Drive/IDS 576/Project/file4.pkl')

In [0]:
pfinal = pd.concat([p1,p2,p3,p4])

In [11]:
len(pfinal)

100000

In [0]:
sample_file = pfinal.head(1000)
sample_file.to_csv('/content/drive/My Drive/IDS 576/Project/sample_file.csv')

# Taking only those categories which have more than 2000 records

In [0]:
subset_data=pfinal.groupby("Category_name").filter(lambda x: len(x)>2000)
#subset_data=pfinal.groupby("Category_name")

In [13]:
len(subset_data)

81689

In [14]:
subset_data['Category_name'].value_counts() #Counting number of categories

TELEPHONIE - GPS                         30245
INFORMATIQUE                             16918
DECO - LINGE - LUMINAIRE                  7157
LIBRAIRIE                                 6123
AUTO - MOTO                               6120
BIJOUX -  LUNETTES - MONTRES              4382
BRICOLAGE - OUTILLAGE - QUINCAILLERIE     3083
MUSIQUE                                   2921
JEUX - JOUETS                             2399
ELECTROMENAGER                            2341
Name: Category_name, dtype: int64

**Now there are 10 Category classes with more than 2000 records**

**Assigning Integer Labels to each category**

In [0]:
CatInt=list(set(subset_data['Category_name']))

In [615]:
CatInt

['BIJOUX -  LUNETTES - MONTRES',
 'LIBRAIRIE',
 'MUSIQUE',
 'ELECTROMENAGER',
 'DECO - LINGE - LUMINAIRE',
 'BRICOLAGE - OUTILLAGE - QUINCAILLERIE',
 'INFORMATIQUE',
 'AUTO - MOTO',
 'JEUX - JOUETS',
 'TELEPHONIE - GPS']

In [0]:
subset_data_temp = subset_data

In [17]:
subset_data.head()

,_id,imgs,category_id,Category_name,imgs_tensor
0,0,[{'picture': b'\xff\xd8\xff\xe0\x00\x10JFIF\x0...,1000010653,TELEPHONIE - GPS,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
1,1,[{'picture': b'\xff\xd8\xff\xe0\x00\x10JFIF\x0...,1000010653,TELEPHONIE - GPS,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
2,2,[{'picture': b'\xff\xd8\xff\xe0\x00\x10JFIF\x0...,1000004079,INFORMATIQUE,"[[[254, 254, 254], [254, 254, 254], [254, 254,..."
3,3,[{'picture': b'\xff\xd8\xff\xe0\x00\x10JFIF\x0...,1000004141,INFORMATIQUE,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
4,4,[{'picture': b'\xff\xd8\xff\xe0\x00\x10JFIF\x0...,1000015539,BRICOLAGE - OUTILLAGE - QUINCAILLERIE,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."


In [0]:
subset_data_temp = subset_data


In [19]:
subset_data.dtypes

_id               int64
imgs             object
category_id       int64
Category_name    object
imgs_tensor      object
dtype: object

In [0]:
subset_data = subset_data.drop(columns='imgs')

### Map Category names to Category numbers

In [0]:

subset_data['Categ Integer'] = subset_data.apply(lambda x: 0 if x['Category_name']==CatInt[0] else 1 if x['Category_name']==CatInt[1] else 2 if x['Category_name']==CatInt[2] else 3 if x['Category_name']==CatInt[3] else 4 if x['Category_name']==CatInt[4] else 5 if x['Category_name']==CatInt[5] else 6 if x['Category_name']==CatInt[6] else 7 if x['Category_name']==CatInt[7] else 8 if x['Category_name']==CatInt[8] else 9 , axis=1)

In [22]:
subset_data.columns

Index(['_id', 'category_id', 'Category_name', 'imgs_tensor', 'Categ Integer'], dtype='object')

In [427]:
subset_data['Category_name'].value_counts().to_frame()

,Category_name
TELEPHONIE - GPS,30245
INFORMATIQUE,16918
DECO - LINGE - LUMINAIRE,7157
LIBRAIRIE,6123
AUTO - MOTO,6120
BIJOUX - LUNETTES - MONTRES,4382
BRICOLAGE - OUTILLAGE - QUINCAILLERIE,3083
MUSIQUE,2921
JEUX - JOUETS,2399
ELECTROMENAGER,2341


# Dividing the Data into Train(80%) and Test(20%)

**Making separate Dataframes for Data and label**

In [0]:
X=subset_data['imgs_tensor'] #Image tensors would be XData

In [0]:
y = subset_data['Categ Integer']

**Spliting Train and Test Data**

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_traindata, X_valdata, y_traindata, y_valdata = train_test_split(X, y, train_size=0.8, random_state=42)

In [432]:
X_traindata.shape

(65351,)

In [433]:
type(X_traindata)

pandas.core.series.Series

In [434]:
y_traindata.value_counts()

9    24230
6    13522
4     5756
7     4926
1     4886
0     3463
5     2419
2     2369
8     1909
3     1871
Name: Categ Integer, dtype: int64

In [0]:
X_traindata = X_traindata.reset_index(drop=True)

y_traindata = y_traindata.reset_index(drop=True)

In [0]:
x_train_combined = pd.concat([X_traindata,y_traindata],axis=1)

In [438]:
x_train_combined.head(3)

,imgs_tensor,Categ Integer
0,"[[[254, 254, 254], [254, 254, 254], [254, 254,...",3
1,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",9
2,"[[[238, 238, 238], [230, 230, 230], [232, 232,...",9


In [440]:
x_train_combined.append(x_train_combined.iloc[0],ignore_index=True)

,imgs_tensor,Categ Integer
0,"[[[254, 254, 254], [254, 254, 254], [254, 254,...",3
1,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",9
2,"[[[238, 238, 238], [230, 230, 230], [232, 232,...",9
3,"[[[254, 255, 255], [254, 255, 255], [254, 255,...",9
4,"[[[251, 253, 252], [252, 254, 253], [252, 254,...",7
...,...,...
65347,"[[[30, 15, 18], [10, 0, 0], [14, 0, 2], [20, 8...",2
65348,"[[[250, 250, 250], [241, 241, 241], [245, 245,...",2
65349,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",0
65350,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",9


In [0]:
d = {'imgs_tensor':[], 'Categ Integer':[]}
x_categ_final = pd.DataFrame(data=d)

short_length=0
long_length=[]
long_label=[]
for i in range(10):
  x_categ_temp = x_train_combined[x_train_combined['Categ Integer']==i]
  x_categ_temp = x_categ_temp.reset_index(drop=True)
  if(len(x_categ_temp) <7000):
    diff = 7000-len(x_categ_temp)
    for j in range(diff):
      x_categ_temp = x_categ_temp.append(x_categ_temp.iloc[j])
    short_length+=len(x_categ_temp)
    x_categ_final = x_categ_final.append(x_categ_temp)
  
  else:
    long_label.append(i)
    long_length.append(len(x_categ_temp))
  
rem_length = 90000-short_length
long_length_total  = sum(long_length)

for i in range(len(long_label)):
  x_categ_temp = x_train_combined[x_train_combined['Categ Integer']==long_label[i]]
  x_categ_temp = x_categ_temp.reset_index(drop=True)
  size = int((long_length[i]/long_length_total)*rem_length)
  x_categ_temp = x_categ_temp[0:size]
  x_categ_final = x_categ_final.append(x_categ_temp)

  
  



In [0]:
x_categ_final_new = x_categ_final.sample(frac=1).reset_index(drop=True)

In [0]:
X_traindata = x_categ_final_new['imgs_tensor']
y_traindata = x_categ_final_new['Categ Integer']

In [0]:
type(X_traindata)
y_traindata = y_traindata.astype(int)

In [620]:
y_traindata.value_counts()

9    21821
6    12178
8     7000
7     7000
5     7000
4     7000
3     7000
2     7000
1     7000
0     7000
Name: Categ Integer, dtype: int64

In [572]:
y_traindata.head(5)

0    9
1    8
2    0
3    6
4    6
Name: Categ Integer, dtype: int64

**Reindexing the Train, Test & Validation Data**

In [0]:
X_traindata = X_traindata.reset_index(drop=True)
X_valdata = X_valdata.reset_index(drop=True)
#X_testdata = X_testdata.reset_index(drop=True)

 **Resizing 180 * 180 Images to 32 * 32 size for train, test & validation Data**

In [0]:
x_train_data = X_traindata.to_numpy()
x_val_data = X_valdata.to_numpy()
#x_test_data = X_testdata.to_numpy()

In [575]:
type(X_traindata)

pandas.core.series.Series

In [0]:
dim = (32,32)

In [0]:
import cv2

X_traindatare = X_traindata.apply(lambda x:cv2.resize(x, dim, interpolation = cv2.INTER_AREA))

In [578]:
type(X_traindatare)

pandas.core.series.Series

In [579]:
X_traindatare.shape

(93752,)

In [0]:
X_valdatare = X_valdata.apply(lambda x:cv2.resize(x, dim, interpolation = cv2.INTER_AREA))

In [582]:
X_valdatare.iloc[4]

array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]]

In [0]:
ftrain = X_traindatare
fval = X_valdatare


In [584]:
type(ftrain)

pandas.core.series.Series

**Converting Train, Test & Validation Data to Numpy Array for Model Input**

In [585]:
type(ftrain)

pandas.core.series.Series

In [586]:
X_train_data = []
for myFile in ftrain:
    X_train_data.append(myFile)

print('X_data shape:', np.array(X_train_data).shape)

X_data shape: (93752, 32, 32, 3)


In [587]:
type(X_train_data)

list

In [0]:
X_train_data2 = np.array(X_train_data)

In [589]:
X_val_data = []
for myFile in fval:
    X_val_data.append(myFile)

print('X_data shape:', np.array(X_val_data).shape)

X_data shape: (16338, 32, 32, 3)


In [0]:
X_val_data2 = np.array(X_val_data)

# Converting ALL Numpy arrays to pytorch tensors

In [0]:
#X_train_data2
#X_val_data2
#X_test_data2

In [0]:
#Function to convert a numpy array to pytorch tensor
def tensor_convert(f_train):
  f_train_tf = torch.from_numpy(f_train)
  f_train_tf = f_train_tf.type('torch.FloatTensor')
  return f_train_tf

In [0]:
x_train_tf = tensor_convert(X_train_data2)
x_val_tf = tensor_convert(X_val_data2)
#x_test_tf = tensor_convert(X_test_data2)

In [0]:
#x_test_tf = tensor_convert(X_test_data2)

In [0]:
l_train_tt = torch.from_numpy(np.array(y_traindata))
l_val_tt = torch.from_numpy(np.array(y_valdata))
#l_test_tt = torch.from_numpy(np.array(y_testdata))

In [0]:

transformations_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

transformations_val = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])



In [0]:
trans = transforms.ToPILImage()

# Modelling using 'Pytorch'

In [598]:
#Vanilla case: Use RESNET with 
def model_initialize():
  #download the Pre-trained RESNET model
  import torchvision.models as models
  model = models.resnet18(pretrained = True)
  
  return model

def model_freeze(model):
  #Freeze the Parameters of RESNET 
  for param in model.parameters():
      param.requires_grad = False 

  return model



def model_last_layer(model):

# Since imagenet as 1000 classes , We need to change our last layer to 10 (number of classes we have)
  num_ftrs = model.fc.in_features
  
  classifier = nn.Sequential(OrderedDict([
      ('fc1', nn.Linear(num_ftrs, 256)),
      ('relu1', nn.ReLU()),
      ('fc2', nn.Linear(256, 128)),
      ('relu2', nn.ReLU()),
      ('fc3', nn.Linear(128,10)),
      ('output', nn.LogSoftmax(dim=1))]))
  model.fc = classifier
  return model




"\n  classifier = nn.Sequential(OrderedDict([\n      ('fc1', nn.Linear(num_ftrs, 10)),\n\n      ('output', nn.LogSoftmax(dim=1))]))\n  model.fc = classifier"

In [0]:
#Initialize Model

model_1 = model_initialize()
model_1 = model_freeze(model_1)
model_1 = model_last_layer(model_1)

In [623]:
print(model_1)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [0]:
criterion = nn.CrossEntropyLoss()
model_1.train()
optimizer = optim.Adam(model_1.fc.parameters(), lr=0.001)
scheduler =  lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [625]:
#Enable CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
use_cuda = True
if use_cuda and torch.cuda.is_available():
  model_1.cuda()

print('Device:', torch.device('cuda:0'))

Device: cuda:0


In [626]:
temp_train = x_train_tf
temp_train.shape

torch.Size([93752, 32, 32, 3])

In [0]:
epochs = 15

In [628]:
for param_group in optimizer.param_groups:
  lr = param_group['lr']
  print("Initial Learning Rate :",lr)

Initial Learning Rate : 0.001


In [606]:
print("Train acc in epoch {} is: {}".format(2,2))

Train acc in epoch 2 is: 2


In [607]:

batch_size =500
model_1.train()
for k in range(epochs):
  running_acc =0 
  if(k==7):
    for param_group in optimizer.param_groups:
      lr = param_group['lr']

    optimizer = optim.SGD(model_1.parameters(), lr=lr,momentum=0.9)    

  
  

  for i in range(int(len(temp_train)/batch_size)):
        optimizer.zero_grad()
        images_list = []
        pre_images = temp_train[batch_size*i:batch_size*i+batch_size]
        for j in range(len(pre_images)):
          images_list.append(transformations_train(trans(pre_images[j])))
        images = torch.stack(images_list)
        labels = l_train_tt[batch_size*i:batch_size*i+batch_size]

        if use_cuda and torch.cuda.is_available():
          images = images.cuda()  
          labels = labels.cuda()
        model_1.to(device)
        log_val = model_1(images)
        value,preds = torch.max(log_val, 1)

        loss = criterion(log_val,labels)
        running_acc+=torch.sum(preds == labels)

        loss.backward()
        optimizer.step()
  scheduler.step()
  print("Train acc in epoch {} is: {}".format(k+1,(running_acc*100)/len(temp_train)))

Train acc in epoch 1 is: 31
Train acc in epoch 2 is: 34
Train acc in epoch 3 is: 35
Train acc in epoch 4 is: 35
Train acc in epoch 5 is: 36
Train acc in epoch 6 is: 37
Train acc in epoch 7 is: 37
Train acc in epoch 8 is: 37
Train acc in epoch 9 is: 37
Train acc in epoch 10 is: 37
Train acc in epoch 11 is: 37
Train acc in epoch 12 is: 37
Train acc in epoch 13 is: 37
Train acc in epoch 14 is: 37
Train acc in epoch 15 is: 37


In [629]:

# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()


Gen RAM Free: 13.4 GB  | Proc size: 19.9 GB
GPU RAM Free: 5893MB | Used: 10387MB | Util  64% | Total 16280MB


In [630]:
#Intialize count: To calculate number of correctly predicted classes
w, h = 1, 10
count = [[0 for x in range(w)] for y in range(h)] 

for i in range(10):
  count[i]=0

print(count)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [0]:

model_1.eval()
batch_size = 100


temp_val = x_val_tf
for i in range(int(len(temp_val)/batch_size)):
  image_list = []
  pre_images = temp_val[batch_size*i:batch_size*i+batch_size]
  for j in range(len(pre_images)):
    image_list.append(transformations_val(trans(pre_images[j])))
  images = torch.stack(image_list)
  
  images = images.cuda()
  labels = l_val_tt[batch_size*i:batch_size*i+batch_size]
  labels = labels.cuda()

  img_val = model_1(images)
  y_pred=[]
  for j in range(len(pre_images)):
    values, index = torch.max(img_val[j],0)
    y_pred.append(index)
    if(y_pred[j]==labels[j]):
      count[labels[j]]+=1
      


In [612]:
print("Validation Accuracy: {}".format((sum(count)*100)/len(x_val_tf)))

Validation Accuracy: 43.58550618190721


In [613]:
for i in range(0,10):
  print(CatInt[i],": ",count[i]/y_valdata.value_counts()[i]*100)


BIJOUX -  LUNETTES - MONTRES :  0.0
LIBRAIRIE :  0.0
MUSIQUE :  65.94202898550725
ELECTROMENAGER :  21.48936170212766
DECO - LINGE - LUMINAIRE :  6.638115631691649
BRICOLAGE - OUTILLAGE - QUINCAILLERIE :  0.0
INFORMATIQUE :  30.624263839811544
AUTO - MOTO :  0.5025125628140703
JEUX - JOUETS :  0.20408163265306123
TELEPHONIE - GPS :  91.70407315045719
